### Imports

### Yield Curves

Yield curves are a snapshot of interest rates in that market—a simple yet often informative graph that plots yields to maturity on the vertical axis and times to maturity on the horizontal axis for a homogeneous set of securities.

The idea of any yield curve in principle is to display visually the relationship between interest rates on securities that are alike on all dimensions except maturity. Ideally, all the observations would be for securities that have the same credit risk, same liquidity, same periodicity, and same tax status. 

Most yield curves are based on government securities. That’s so we can hold constant all the listed factors other than time to maturity that impact investors’ required rates of return. The yields to maturity ideally would be for zero‐coupon securities so that coupon reinvestment risk is not a factor.

There is no perfect data set for term structure analysis. Typically seen yield curves are plots of street convention yields on coupon-bearing Treasury notes and bonds instead of yields on Treasury STRIPS. In particular, the yields displayed usually are for on‐the‐run issues (i.e., the most recently auctioned Treasury securities). These are actively traded and typically are priced close to par value, thereby minimizing the effects of the deferral for tax purposes of the gains and losses from buying at a discount or a premium. The problem is that there are gaps in the times to maturity, so some yields have to be interpolated.

### Implied Forward Curve